In [1]:
import logging
from logging.handlers import RotatingFileHandler
import requests
import pandas as pd
import os

# Create the directory if it doesn't exist
log_dir = r'C:\Users\mohan\OneDrive\Documents\MS\CGS 2100\POC\log'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Configure logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

# Create a file handler
file_handler = RotatingFileHandler(os.path.join(log_dir, 'api_log.log'), maxBytes=100000, backupCount=3)
file_handler.setLevel(logging.DEBUG)
file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))

# Add the file handler to the root logger
logging.getLogger().addHandler(file_handler)

def fetch_data(api_url, headers):
    response = requests.get(api_url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        logging.error(f"Failed to fetch data from API. Status code: {response.status_code}")
        return None

def get_all_data(api_url, headers):
    all_data = []
    api_calls = 0  # Initialize counter for API calls
    while True:  # Continue looping until there are no more records
        response = requests.get(api_url, headers=headers)
        api_calls += 1  # Increment API call counter
        if response.status_code == 200:
            data = response.json()
            all_data.extend(data)
            if 'next' in response.links:
                next_url = response.links['next']['url']
                api_url = next_url
            else:
                logging.info("No more records to retrieve. Stopping pagination.")
                break  # Exit the loop if there are no more pages
        else:
            logging.error(f"Failed to fetch data from API. Status code: {response.status_code}")
            break  # Exit the loop if the API request fails
    logging.info(f"API called {api_calls} times")  # Log API call count
    return all_data


def main():
    # Define API URL and bearer token
    api_url = "https://usflearn.instructure.com/api/v1/courses/1857108/enrollments"
    bearer_token = "13~D9Rs8zX2eEumEXC0knPDXdIXQLbq4cKVwVnYGmBYEgBFPMI2Yu1suCAk1l1E90Uq"

    # Define headers with bearer token authentication
    headers = {"Authorization": f"Bearer {bearer_token}"}

    # Fetch all data from the API
    all_data = get_all_data(api_url, headers)

    # Convert data to DataFrame
    if all_data:
        df = pd.DataFrame(all_data)
        logging.info("Data fetched successfully and saved to DataFrame.")

        # Save DataFrame to Excel file
        file_path = r"C:\Users\mohan\OneDrive\Documents\MS\CGS 2100\POC\enrollments_data1.xlsx"
        df.to_excel(file_path, index=False)  # Index=False to exclude row numbers

        logging.info(f"Data saved to '{file_path}'.")
        print(df.head())  # Display the first few rows of the DataFrame
    else:
        logging.warning("No data fetched from the API.")

if __name__ == "__main__":
    main()

2024-04-18 15:40:28,463 - DEBUG - Starting new HTTPS connection (1): usflearn.instructure.com:443
2024-04-18 15:40:30,924 - DEBUG - https://usflearn.instructure.com:443 "GET /api/v1/courses/1857108/enrollments HTTP/1.1" 200 None
2024-04-18 15:40:30,932 - DEBUG - Starting new HTTPS connection (1): usflearn.instructure.com:443
2024-04-18 15:40:32,473 - DEBUG - https://usflearn.instructure.com:443 "GET /api/v1/courses/1857108/enrollments?page=bookmark:WyJTdHVkZW50RW5yb2xsbWVudCIsIkFkZWxtYW4sIFRheWxvciIsNDIyODUyMjNd&per_page=10 HTTP/1.1" 200 None
2024-04-18 15:40:32,489 - DEBUG - Starting new HTTPS connection (1): usflearn.instructure.com:443
2024-04-18 15:40:33,932 - DEBUG - https://usflearn.instructure.com:443 "GET /api/v1/courses/1857108/enrollments?page=bookmark:WyJTdHVkZW50RW5yb2xsbWVudCIsIkFsbW9uLCBKYWlkZW4iLDQyMjYzMDM5XQ&per_page=10 HTTP/1.1" 200 None
2024-04-18 15:40:33,943 - DEBUG - Starting new HTTPS connection (1): usflearn.instructure.com:443
2024-04-18 15:40:35,246 - DEBUG - h

         id  user_id  course_id                type            created_at  \
0  42697283  4638795    1857108  DesignerEnrollment  2024-01-08T20:22:12Z   
1  42066599  4959011    1857108   StudentEnrollment  2023-10-31T05:42:10Z   
2  42337449  4940043    1857108   StudentEnrollment  2023-11-19T06:42:02Z   
3  42449855  4836803    1857108   StudentEnrollment  2023-12-08T20:23:59Z   
4  42050207  4853363    1857108   StudentEnrollment  2023-10-30T23:26:20Z   

             updated_at associated_user_id start_at end_at  course_section_id  \
0  2024-01-08T20:37:03Z               None     None   None            2039727   
1  2023-12-12T19:57:37Z               None     None   None            2039735   
2  2023-12-12T19:50:37Z               None     None   None            2039733   
3  2023-12-12T19:53:30Z               None     None   None            2039741   
4  2023-12-12T19:50:37Z               None     None   None            2039733   

   ...  total_activity_time  sis_account_id   sis_

In [4]:
import pandas as pd
from ast import literal_eval
# Load the data from the Excel file
df = pd.read_excel(r'C:\Users\mohan\OneDrive\Documents\MS\CGS 2100\POC\enrollments_data1.xlsx')  # Replace with the path to your file
# Convert total activity time from seconds to hours
df['total_activity_time(in_hrs)'] = df['total_activity_time'] / 3600
# Define the columns to keep
columns_to_keep = [
    'user_id', 'type', 'role', 'last_activity_at',
    'total_activity_time(in_hrs)', 'sis_course_id', 'sis_section_id', 'sis_user_id', 'grades'
]
# Keep only the specified columns
df = df[columns_to_keep]
df.rename(columns={'user_id': 'Student ID'}, inplace=True)
# Check if 'grade' column exists to avoid KeyErrors
if 'grades' in df.columns:
    # Extract information from the 'grade' column if it contains a stringified dictionary
    df['grades'] = df['grades'].apply(lambda x: literal_eval(x) if not pd.isnull(x) and isinstance(x, str) else {})
    # Create new columns for each grade component
    grade_components = ['current_grade', 'current_score', 'final_grade', 'final_score',
                        'unposted_current_score', 'unposted_current_grade',
                        'unposted_final_score', 'unposted_final_grade']   
    for component in grade_components:
        df[component] = df['grades'].apply(lambda x: x.get(component))
    # Now you can drop the original 'grade' column
    df.drop('grades', axis=1, inplace=True)
# Save the cleaned DataFrame to a new Excel file
df.to_excel(r'C:\Users\mohan\OneDrive\Documents\MS\CGS 2100\POC\Processed Files\cleaned_enrollments_data.xlsx', index=False)  # Replace with your desired output path
# Print the first few rows to check the DataFrame
df.head()

,Student ID,type,role,last_activity_at,total_activity_time(in_hrs),sis_course_id,sis_section_id,sis_user_id,current_grade,current_score,final_grade,final_score,unposted_current_score,unposted_current_grade,unposted_final_score,unposted_final_grade
0,4638795,DesignerEnrollment,SAS Assistant,2024-01-10T17:30:48Z,0.117500,CGS2100.001S24,CGS2100.001S24,U64940243,None,NaN,None,NaN,NaN,None,NaN,None
1,4959011,StudentEnrollment,StudentEnrollment,2024-04-17T22:14:55Z,1.897500,CGS2100.001S24,CGS2100.008S24,U87546613,None,99.68,None,89.59,99.68,None,89.59,None
2,4940043,StudentEnrollment,StudentEnrollment,2024-04-18T18:40:15Z,8.936667,CGS2100.001S24,CGS2100.006S24,U30443576,None,97.31,None,87.58,97.31,None,87.58,None
3,4836803,StudentEnrollment,StudentEnrollment,2024-04-17T17:44:00Z,7.833056,CGS2100.001S24,CGS2100.007S24,U97733899,None,83.19,None,74.87,83.19,None,74.87,None
4,4853363,StudentEnrollment,StudentEnrollment,2024-04-18T16:57:59Z,4.270000,CGS2100.001S24,CGS2100.006S24,U67196743,None,101.23,None,91.11,101.23,None,91.11,None


In [5]:
import pandas as pd
from ast import literal_eval
from datetime import datetime

# Load the data from the Excel file
df = pd.read_excel(r'C:\Users\mohan\OneDrive\Documents\MS\CGS 2100\POC\enrollments_data1.xlsx')  # Replace with the path to your file

# Convert 'last_activity_at' to datetime and make it timezone-naive
df['last_activity_at'] = pd.to_datetime(df['last_activity_at']).dt.tz_localize(None)

# Get the current date as timezone-naive
current_date = pd.to_datetime('today').tz_localize(None)

# Calculate inactivity days
df['inactive_days'] = (current_date - df['last_activity_at']).dt.days

# Replace negative values with 0 if 'last_activity_at' happens to be a future date
df['inactive_days'] = df['inactive_days'].apply(lambda x: max(x, 0))

# Convert total activity time from seconds to hours
df['total_activity_time(in_hrs)'] = df['total_activity_time'] / 3600

# Define the columns to keep
columns_to_keep = [
    'user_id', 'type', 'role', 'last_activity_at',
    'total_activity_time(in_hrs)', 'sis_course_id', 'sis_section_id', 'sis_user_id', 'grades','inactive_days'
]

# Keep only the specified columns
df = df[columns_to_keep]

# Rename the user_id column to Student ID
df.rename(columns={'user_id': 'Student ID'}, inplace=True)

# Process grades column if it exists
if 'grades' in df.columns:
    df['grades'] = df['grades'].apply(lambda x: literal_eval(x) if not pd.isnull(x) and isinstance(x, str) else {})
    
    grade_components = ['current_grade', 'current_score', 'final_grade', 'final_score',
                        'unposted_current_score', 'unposted_current_grade',
                        'unposted_final_score', 'unposted_final_grade']
    
    for component in grade_components:
        df[component] = df['grades'].apply(lambda x: x.get(component))
    
    # Drop the original 'grades' column
    df.drop('grades', axis=1, inplace=True)

# Save the cleaned DataFrame to a new Excel file
df.to_excel(r'C:\Users\mohan\OneDrive\Documents\MS\CGS 2100\POC\Processed Files\cleaned_enrollments_data.xlsx', index=False)

# Print the first few rows to verify
print(df.head())


   Student ID                type               role    last_activity_at  \
0     4638795  DesignerEnrollment      SAS Assistant 2024-01-10 17:30:48   
1     4959011   StudentEnrollment  StudentEnrollment 2024-04-17 22:14:55   
2     4940043   StudentEnrollment  StudentEnrollment 2024-04-18 18:40:15   
3     4836803   StudentEnrollment  StudentEnrollment 2024-04-17 17:44:00   
4     4853363   StudentEnrollment  StudentEnrollment 2024-04-18 16:57:59   

   total_activity_time(in_hrs)   sis_course_id  sis_section_id sis_user_id  \
0                     0.117500  CGS2100.001S24  CGS2100.001S24   U64940243   
1                     1.897500  CGS2100.001S24  CGS2100.008S24   U87546613   
2                     8.936667  CGS2100.001S24  CGS2100.006S24   U30443576   
3                     7.833056  CGS2100.001S24  CGS2100.007S24   U97733899   
4                     4.270000  CGS2100.001S24  CGS2100.006S24   U67196743   

   inactive_days current_grade  current_score final_grade  final_score  \
